In [1]:
#Loading packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
import re
#import scrape_functions_un as kzd
import scraping_model as sm 

## Players Dictionary

In [2]:
players_names = {'Rory McIlroy': '17419',
            'Tiger Woods': '10085', 
            'Jordan Spieth': '19976',
            'Justin Thomas': '20037',
            'Scottie Scheffler': '25786',
            'Collin Morikawa': '33992',
            'Sergio García': '10170',
            'Dustin Johnson': '17976',
            'Phil Mickelson': '10083',
            'Ian Poulter': '11956',
            'Talor Gooch': '27168',
            'Cameron Smith': '27755',
            'Joaquin Niemann': '33997',
            'Bryson DeChambeau': '29148',
            'Brooks Koepka': '25730',
            'Jon Rahm': '28177'}

### We want to get all the dates and urls for each of the player's pages

In [48]:
niemann_url = 'http://www.asapsports.com/show_player.php?id=33997'

response = requests.get(niemann_url)
soup = BeautifulSoup(response.text, 'html.parser')
dates_niemann = soup.find_all('b')

In [49]:
## Retrieving player interview links
links_niemann = []
for row in dates_niemann:
        a_tags = soup.find_all('a', href=True)
        
for atag in a_tags:
    url = atag['href']
    if url.startswith('http://www.asapsports.com/show_interview.php?id='):
         links_niemann.append(url)

### Doing a for loop, to get all the links for each player

In [10]:
links = {}
for player, id in players_names.items():
    player_url = f'https://www.asapsports.com/show_player.php?id={id}'
    response = requests.get(player_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dates_links = soup.find_all('b')
    links_player = []

    for row in dates_links:
            a_tags = soup.find_all('a', href=True)
        
    for atag in a_tags:
        url = atag['href']
        if url.startswith('http://www.asapsports.com/show_interview.php?id='):
            links_player.append(url)
    links[f'{player}'] = links_player
    print('URLs fetched from:', player)

#3m 30.3s    

URLs fetched from: Rory McIlroy
URLs fetched from: Tiger Woods
URLs fetched from: Jordan Spieth
URLs fetched from: Justin Thomas
URLs fetched from: Scottie Scheffler
URLs fetched from: Collin Morikawa
URLs fetched from: Sergio García
URLs fetched from: Dustin Johnson
URLs fetched from: Phil Mickelson
URLs fetched from: Ian Poulter
URLs fetched from: Talor Gooch
URLs fetched from: Cameron Smith
URLs fetched from: Joaquin Niemann
URLs fetched from: Bryson DeChambeau
URLs fetched from: Brooks Koepka
URLs fetched from: Jon Rahm


## Big loop, for all the links for all the players

In [43]:
press_conferences = pd.DataFrame(columns=['date','question','answer'])

events = []

for player, link_list in links.items():
    for link in link_list:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Getting Data
        event = soup.find('h1').text
        events.append(event)

        dates_loop, answers_loop = sm.parse_interview(link)

        bold_text = soup.find_all('b')

        questions = sm.get_questions(bold_text) 
        stripped_questions = [question.strip('Q.') for question in questions]
        stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

        print(len(answers_loop), '\t', len(stripped_questions))




        num_to_fill = len(answers_loop) - len(stripped_questions)
        stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions
        # print(len(dates_loop))
        # print(len(stripped_questions))
        # print(answers_loop[0])

        # temp_df = pd.DataFrame({'date': dates_loop, 'answer': answers_loop, 'question': stripped_questions})
        # press_conferences = pd.concat([press_conferences, temp_df], ignore_index=True)
        # print('Press conference from:',event,'\t by:',player)

1 	 4
1 	 6
1 	 18
1 	 4
1 	 5
3 	 12
1 	 5
2 	 17
1 	 4


KeyboardInterrupt: 

In [10]:

# URL = 'https://www.asapsports.com/show_interview.php?id=194453'
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
# response = requests.get(URL, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')

# # Getting Data
# event = soup.find('h1')
# dates = soup.find('h2')
# names_soup = soup.find_all('h3')


# names = []
# for i in names_soup:
#     if i.text in players_names:
#         names.append(i.text)


# print(dates.text)
# print(event.text)
# print(names)


In [7]:
# dates, answers = sm.parse_interview(URL)
# niemann_result = pd.DataFrame({'date':dates,'answer':answers})

In [11]:
# niemann_result

In [12]:
# niemann_result['player'] = [i for i, n in niemann_result['answer']]
# niemann_result['answer'] = [n for i, n in niemann_result['answer']]
# niemann_result

In [14]:
# bold_text = soup.find_all('b')
# questions = get_questions(bold_text) 
# stripped_questions = [question.strip('Q.') for question in questions]
# stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

# # Si no hay pregunta para la respuesta, quiero que rellenes con 'MODERATOR' 

since the moderator questions don't have a fixed pattern, we struggle to retrieve them, we will impute them with 'moderator' in the question; in the text analysis we might get rid of the token so it doesn't consider it an important token.

In [13]:
# num_to_fill = len(niemann_result['answer']) - len(stripped_questions)
# stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions

# niemann_result['question'] = stripped_questions

In [48]:
# niemann_result['Ans LIV?'] =niemann_result['answer'].apply(lambda x: 1 if 'liv' in x else 0)
# niemann_result['Ans PGA?'] =niemann_result['answer'].apply(lambda x: 1 if 'pga' in x else 0)
# niemann_result['Q LIV?'] =niemann_result['question'].apply(lambda x: 1 if 'liv' in x else 0)
# niemann_result['Q PGA?'] =niemann_result['question'].apply(lambda x: 1 if 'pga' in x else 0)


In [15]:
#niemann_result

### For loop for all the links in the players page

In [50]:
press_conferences = pd.DataFrame(columns=['date','question','answer'])
events = []
for link in links_niemann:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Getting Data
    event = soup.find('h1').text
    events.append(event)

    dates_loop, answers_loop = sm.parse_interview(link)

    bold_text = soup.find_all('b')
    questions = sm.get_questions(bold_text) 
    stripped_questions = [question.strip('Q.') for question in questions]
    stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

    print(answers_loop)

    # num_to_fill = len(answers_loop) - len(stripped_questions)
    # stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions
    # temp_df = pd.DataFrame({'date': dates_loop, 'answer': answers_loop, 'question': stripped_questions})
    # press_conferences = pd.concat([press_conferences, temp_df], ignore_index=True)
    print('Press conference from:',event)






[('joaquin niemann', ['yeah', 'it', 'feels', 'amazing', "it's", 'a', 'super', 'special', 'moment', "it's", 'always', 'nice', 'when', 'you', 'win', "i'm", 'just', 'more', 'happy', "i'm", 'more', 'proud', 'the', 'way', "i'm", 'playing', 'right', 'now', 'and', 'the', 'way', 'i', 'played', 'out', 'there', 'today', 'it', 'was', 'a', 'tough', 'day', 'it', 'was', 'windy', 'i', 'knew', 'there', 'was', 'going', 'to', 'be', 'some', 'low', 'scores', 'so', "i'm", 'super', 'happy', "i'm", 'proud', 'the', 'way', "i'm", 'playing', 'i', 'just', 'want', 'to', 'on', 'to', 'the', 'next', 'one']), ('joaquín niemann', ['i', "wouldn't", 'say', 'it', 'felt', 'that', 'way', 'at', 'the', 'moment', 'there', 'were', 'some', 'tricky', 'shots', 'out', 'there', 'there', 'were', 'some', 'tough', 'holes', 'especially', 'with', 'this', 'wind', 'last', 'couple', 'holes', 'you', 'have', "it's", 'a', 'good', 'finish', 'and', 'also', 'started', 'to', 'feel', 'some', 'pressure', 'there', 'and', 'being', 'able', 'to', 'bird

KeyboardInterrupt: 

In [37]:
press_conferences2 = press_conferences.copy()

In [39]:
press_conferences.head()

,date,question,answer
0,"March 3, 2024",MODERATOR,"(joaquin niemann, [yeah, it, feels, amazing, i..."
1,"March 3, 2024","[you, looked, in, control, out, there, all, da...","(joaquín niemann, [i, wouldn't, say, it, felt,..."
2,"March 3, 2024","[the, way, you're, playing, right, now, do, yo...","(joaquin niemann, [i, mean, at, least, that's,..."
3,"March 3, 2024","[is, this, as, good, of, a, confidence, level,...","(joaquin niemann, [i, believe, so, yeah, i, fe..."
4,"March 3, 2024","[there, was, one, shot, i, think, it, was, the...","(joaquin niemann, [oh, that, was, beautiful, y..."


In [7]:
press_conferences2['player'] = [i for i, n in press_conferences2['answer']]
press_conferences2['answer'] = [n for i, n in press_conferences2['answer']]

# press_conferences2['Ans LIV?'] =press_conferences2['answer'].apply(lambda x: 1 if 'liv' in x else 0)
# press_conferences2['Ans PGA?'] =press_conferences2['answer'].apply(lambda x: 1 if 'pga' in x else 0)
# press_conferences2['Q LIV?'] =press_conferences2['question'].apply(lambda x: 1 if 'liv' in x else 0)
# press_conferences2['Q PGA?'] =press_conferences2['question'].apply(lambda x: 1 if 'pga' in x else 0)

In [8]:
press_conferences2

,date,question,answer,player
0,"March 3, 2024",MODERATOR,"[yeah, it, feels, amazing, it's, a, super, spe...",joaquin niemann
1,"March 3, 2024","[you, looked, in, control, out, there, all, da...","[i, wouldn't, say, it, felt, that, way, at, th...",joaquín niemann
2,"March 3, 2024","[the, way, you're, playing, right, now, do, yo...","[i, mean, at, least, that's, what, i, think, i...",joaquin niemann
3,"March 3, 2024","[is, this, as, good, of, a, confidence, level,...","[i, believe, so, yeah, i, feel, like, it's, pr...",joaquin niemann
4,"March 3, 2024","[there, was, one, shot, i, think, it, was, the...","[oh, that, was, beautiful, yeah, that, hole, i...",joaquin niemann
...,...,...,...,...
1060,"August 15, 2017",MODERATOR,"[i, think, i, feel, better, in, riviera, than,...",joaquin niemann
1061,"August 14, 2017",MODERATOR,"[under, yeah]",joaquin niemann
1062,"August 14, 2017",MODERATOR,"[yeah, i, played, really, good, golf, the, gol...",joaquin niemann
1063,"August 14, 2017",MODERATOR,"[i, like, the, course, i, think, it's, more, l...",joaquin niemann


## TENEMOS UN PROBLEMA, HAY CONFERENCIAS QUE TIENEN MAS DE UN JUGADOR RESPONDIENDO. ESTÁN INTERCALADAS LAS PREGUNTAS ENTONCES TENEMOS QUE ASIGNAR EL JUGADOR DEPENDIENDO DE LA RTA Y NO DE LA CONFERENCIA EN GRAL.

## HAY QUE PILLAR LAS FECHAS PARA LUEGO FILTRAR A PARTIR DE OCTUBRE DE 2020

### Creo que hay que sacar todos los parrafor y luego hacer functiones que diferencien entre negrita y no. Que extraigan el autor si empieza en mayúscula y luego dos puntos... Habrá que checkear también si quien habla aparece dentro de los nombres en la conferencia de prensa, y luego checkear si es uno de nuestros jugadores seleccionados.
### También hay que hacer una lista con jugadores por conferencia en vez de un único valor